In [1]:
import sqlite3
import os
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import cartopy.crs as ccrs
#import seaborn as sns
import re
from datetime import datetime

In [2]:
# walk around to look for local txt files 
path = './'
infofiles = []
for dirpath, subdirs, files in os.walk(path):
    for x in files:
        if x.endswith("INFO.txt") :
            infofiles.append(os.path.join(dirpath, x))
            
infofiles

['./data/20140101k_stemp1_INFO.txt',
 './data/20180101s_stemp6_INFO.txt',
 './data/20170101s_stemp1_INFO.txt',
 './data/20190101s_stemp6_INFO.txt',
 './data/20190101k_stemp1_INFO.txt',
 './data/20160101w_wave_INFO.txt',
 './data/20160101v_wind_INFO.txt',
 './data/20190101v_wind_INFO.txt',
 './data/20190101s_stemp1_INFO.txt',
 './data/20190101w_wave_INFO.txt',
 './data/20170101v_wind_INFO.txt',
 './data/20140101p_pres1_INFO.txt',
 './data/20180101k_stemp1_INFO.txt',
 './data/20160101s_stemp1_INFO.txt',
 './data/20150101v_wind_INFO.txt',
 './data/20160101p_pres1_INFO.txt',
 './data/20180101v_wind_INFO.txt',
 './data/20170101w_wave_INFO.txt',
 './data/20150101p_pres1_INFO.txt',
 './data/20130101v_wind_INFO.txt',
 './data/20140101v_wind_INFO.txt',
 './data/20130101w_wave_INFO.txt',
 './data/20170101k_stemp1_INFO.txt',
 './data/20150101k_stemp1_INFO.txt',
 './data/20180101w_wave_INFO.txt',
 './data/20130101p_pres1_INFO.txt',
 './data/20180101p_pres1_INFO.txt',
 './data/20150101s_stemp1_INFO

#### take care of big5 and utf8

In [3]:
def convert_big52utf8(filename):
    fh = open(filename,'r',encoding="big5")
    _lines = fh.readlines()
    fh.close()
    _lines

    filename=filename+'.utf8'
    fh = open(filename,'w',encoding="utf-8")
    fh.writelines(_lines)
    fh.close()

In [4]:
#filename='./20170101s_stemp6_INFO.txt'
#convert_big52utf8(filename)

In [8]:
special=['4N11','FJ8200','FJ8201','FJ8202','FJ8203','FJ8205','FJB022','FJB023','FJB085','FJB086','FJB087','FJ0046','19901']

def info(line):
    data_dict={}
    data_dict['STN'] = line[1]
    data_dict['LAT'] = line[2]
    data_dict['LON'] = line[3]
    data_dict['LOCATE'] = line[4] 
    data_dict['OWNER'] = line[5] 
    return data_dict
    
stndata={}
for f,file in enumerate(infofiles):
    # data filename 
    #print(file)
    fh = open(file,'r')
    _lines = fh.readlines()
    fh.close()

    years=[2014,2018,2017,2016,2015,2013]
    if int(file.lstrip('./data/')[0:4]) in years:
        if _lines[1][0] == '*':
            _lines.pop(1)
        for line in _lines[2:]:
            v = [var.strip() for var in line.strip().split(' ')]
            while '' in v: v.remove('')
            if v[0] in special:
                v.append(v[2])
                v.append(v[3])
                v[2] = ''
                v[3] = ''
            
            if len(v) < 6:
                print(v)
                if v[0][0] == 'C'  :
                    v.append(v[4])
                    v[4] = ''
                else:
                    v.append('')
                
            stndata[v[0]] = info(v)
    else:
        for line in _lines[2:]:
            v = [var.strip() for var in line.strip().split(',')]
            stndata[v[0]] = info(v)
            if len(v) == 5:
                print(v)
stndata

['198', '東沙島', '20°42\'00"N', '116°41\'24"E', '內政部']
['FJ0046', '福建古雷小浮標', '', '', '古雷']
['FJ8200', '福建北礵小浮標', '', '', '北礵']
['FJ8201', '福建黃岐小浮標', '', '', '黃岐']
['FJ8202', '福建斗尾港小浮標', '', '', '斗尾港']
['FJ8203', '福建平潭小浮標', '', '', '平潭']
['FJ8205', '福建牛山島小浮標', '', '', '牛山島']
['C0A88', '福隆', '25°01\'09"N', '121°56\'03"E', '氣象局']
['C0A92', '富貴角', '25°15\'56"N', '121°33\'26"E', '氣象局']
['C0A95', '鼻頭角', '25°07\'03"N', '121°53\'55"E', '氣象局']
['FJB022', '福建大浮標2號', '', '', '泉州外海']
['FJB023', '福建大浮標4號', '', '', '羅源外海']
['FJB085', '福建大浮標1號', '', '', '東山外海']
['FJB086', '福建大浮標3號', '', '', '閩江口外海']
['FJB087', '福建大浮標5號', '', '', '寧德外海']
['C0C57', '新屋自動氣象站', '25°0\'57"N', '121°2\'36"E', '氣象局']
['C0C59', '觀音', '25°01\'30"N', '121°09\'41"E', '氣象局']
['C0D57', '香山', '24°44\'48"N', '121°54\'19"E', '氣象局']
['C0D59', '新豐', '24°52\'07"N', '120°59\'7"E', '氣象局']
['C0E42', '竹南', '24°42\'38"N', '120°52\'51"E', '氣象局']
['C0G64', '鹿港', '24°04\'37"N', '120°25\'20"E', '氣象局']
['C0R27', '琉球嶼', '22°19\'51"N', '120°20\'48"E'

{'4N11': {'STN': '將軍',
  'LAT': '',
  'LON': '',
  'LOCATE': '臺南將軍安檢所樓頂',
  'OWNER': '氣象局'},
 '46694A': {'STN': '龍洞浮標',
  'LAT': '25°05\'52"N',
  'LON': '121°55\'21"E',
  'LOCATE': '東北角龍洞遊艇港外海約0.25公里，該處水深約27公尺。',
  'OWNER': '中央氣象局'},
 '46699A': {'STN': '花蓮浮標',
  'LAT': '24°01\'55"N',
  'LON': '121°37\'53"E',
  'LOCATE': '花蓮七星潭外海約1公里，該處水深約30公尺。',
  'OWNER': '中央氣象局'},
 '46708A': {'STN': '龜山島浮標',
  'LAT': '24°50\'48"N',
  'LON': '121°55\'34"E',
  'LOCATE': '宜蘭龜山島龜尾端西方1公里，該處水深約25公尺。',
  'OWNER': '交通部觀光局'},
 '46714D': {'STN': '小琉球浮標',
  'LAT': '22°18\'49"N',
  'LON': '120°21\'46"E',
  'LOCATE': '小琉球海子口西南方外海約1.2公里，該處水深約99公尺。',
  'OWNER': '中央氣象局'},
 '46757B': {'STN': '新竹浮標',
  'LAT': '24°45\'38"N',
  'LON': '120°50\'46"E',
  'LOCATE': '新竹海山漁港外海離岸約6公里，該處水深約25公尺。',
  'OWNER': '中央氣象局'},
 'C6S62': {'STN': '臺東外洋浮標',
  'LAT': '21°45\'59"N',
  'LON': '124°04\'27"E',
  'LOCATE': '臺東東南方外海約300公里，該處水深約5610公尺。',
  'OWNER': '氣象局'},
 'C6V27': {'STN': '東沙島浮標',
  'LAT': '21°04\'24"N',
  'LON': '118°49\'12"E'

In [9]:
def latlon_str2float(string):
    if string == '':
        number = 'null'
        return str(number)
    else:
        print(string)

    d1 = string.split('\'')[0]
    d2 = string.split('\'')[1]
    s1 = float(d1.split('°')[0])
    s2 = float(d1.split('°')[1])/60.
    c1 = float(d2.split('"')[0])/3600.
    c2 = d2.split('"')[1]
    print(s1,s2,c1,c2)
    number = s1+s2+c1
    return number

In [10]:
for data in stndata:
    print(stndata[data]['LAT'])
    stndata[data]['LAT'] = latlon_str2float(stndata[data]['LAT'])
    stndata[data]['LON'] = latlon_str2float(stndata[data]['LON'])


25°05'52"N
25°05'52"N
25.0 0.08333333333333333 0.014444444444444444 N
121°55'21"E
121.0 0.9166666666666666 0.005833333333333334 E
24°01'55"N
24°01'55"N
24.0 0.016666666666666666 0.015277777777777777 N
121°37'53"E
121.0 0.6166666666666667 0.014722222222222222 E
24°50'48"N
24°50'48"N
24.0 0.8333333333333334 0.013333333333333334 N
121°55'34"E
121.0 0.9166666666666666 0.009444444444444445 E
22°18'49"N
22°18'49"N
22.0 0.3 0.01361111111111111 N
120°21'46"E
120.0 0.35 0.012777777777777779 E
24°45'38"N
24°45'38"N
24.0 0.75 0.010555555555555556 N
120°50'46"E
120.0 0.8333333333333334 0.012777777777777779 E
21°45'59"N
21°45'59"N
21.0 0.75 0.01638888888888889 N
124°04'27"E
124.0 0.06666666666666667 0.0075 E
21°04'24"N
21°04'24"N
21.0 0.06666666666666667 0.006666666666666667 N
118°49'12"E
118.0 0.8166666666666667 0.0033333333333333335 E
26°21'9"N
26°21'9"N
26.0 0.35 0.0025 N
120°30'55"E
120.0 0.5 0.015277777777777777 E
25°11'3"N
25°11'3"N
25.0 0.18333333333333332 0.0008333333333333334 N
121°24'25"

In [11]:
stndata

{'4N11': {'STN': '將軍',
  'LAT': 'null',
  'LON': 'null',
  'LOCATE': '臺南將軍安檢所樓頂',
  'OWNER': '氣象局'},
 '46694A': {'STN': '龍洞浮標',
  'LAT': 25.097777777777775,
  'LON': 121.9225,
  'LOCATE': '東北角龍洞遊艇港外海約0.25公里，該處水深約27公尺。',
  'OWNER': '中央氣象局'},
 '46699A': {'STN': '花蓮浮標',
  'LAT': 24.031944444444445,
  'LON': 121.63138888888888,
  'LOCATE': '花蓮七星潭外海約1公里，該處水深約30公尺。',
  'OWNER': '中央氣象局'},
 '46708A': {'STN': '龜山島浮標',
  'LAT': 24.846666666666664,
  'LON': 121.92611111111111,
  'LOCATE': '宜蘭龜山島龜尾端西方1公里，該處水深約25公尺。',
  'OWNER': '交通部觀光局'},
 '46714D': {'STN': '小琉球浮標',
  'LAT': 22.31361111111111,
  'LON': 120.36277777777777,
  'LOCATE': '小琉球海子口西南方外海約1.2公里，該處水深約99公尺。',
  'OWNER': '中央氣象局'},
 '46757B': {'STN': '新竹浮標',
  'LAT': 24.760555555555555,
  'LON': 120.8461111111111,
  'LOCATE': '新竹海山漁港外海離岸約6公里，該處水深約25公尺。',
  'OWNER': '中央氣象局'},
 'C6S62': {'STN': '臺東外洋浮標',
  'LAT': 21.76638888888889,
  'LON': 124.07416666666666,
  'LOCATE': '臺東東南方外海約300公里，該處水深約5610公尺。',
  'OWNER': '氣象局'},
 'C6V27': {'STN': '東沙島浮標'

In [48]:
import json
with open('buoy_station_name.json','w',encoding='utf8') as outfile:
    json.dump(stndata,outfile, ensure_ascii=False, indent=4)

In [49]:
stndata['4N11']

{'STN': '將軍',
 'LAT': 'null',
 'LON': 'null',
 'LOCATE': '臺南將軍安檢所樓頂',
 'OWNER': '氣象局'}

In [50]:
with open('buoy_station_name.json','r') as outfile:
    data=json.load(outfile)
data

{'4N11': {'STN': '將軍',
  'LAT': 'null',
  'LON': 'null',
  'LOCATE': '臺南將軍安檢所樓頂',
  'OWNER': '氣象局'},
 '46694A': {'STN': '龍洞浮標',
  'LAT': 25.097777777777775,
  'LON': 121.9225,
  'LOCATE': '東北角龍洞遊艇港外海約0.25公里，該處水深約27公尺。',
  'OWNER': '中央氣象局'},
 '46699A': {'STN': '花蓮浮標',
  'LAT': 24.031944444444445,
  'LON': 121.63138888888888,
  'LOCATE': '花蓮七星潭外海約1公里，該處水深約30公尺。',
  'OWNER': '中央氣象局'},
 '46708A': {'STN': '龜山島浮標',
  'LAT': 24.846666666666664,
  'LON': 121.92611111111111,
  'LOCATE': '宜蘭龜山島龜尾端西方1公里，該處水深約25公尺。',
  'OWNER': '交通部觀光局'},
 '46714D': {'STN': '小琉球浮標',
  'LAT': 22.31361111111111,
  'LON': 120.36277777777777,
  'LOCATE': '小琉球海子口西南方外海約1.2公里，該處水深約99公尺。',
  'OWNER': '中央氣象局'},
 '46757B': {'STN': '新竹浮標',
  'LAT': 24.760555555555555,
  'LON': 120.8461111111111,
  'LOCATE': '新竹海山漁港外海離岸約6公里，該處水深約25公尺。',
  'OWNER': '中央氣象局'},
 'C6S62': {'STN': '臺東外洋浮標',
  'LAT': 21.76638888888889,
  'LON': 124.07416666666666,
  'LOCATE': '臺東東南方外海約300公里，該處水深約5610公尺。',
  'OWNER': '氣象局'},
 'C6V27': {'STN': '東沙島浮標'